In [ ]:
pip install mysql-connector-python

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
import pandas as pd
import pyodbc
import traceback

# Function to read database configuration from a file
def read_db_config(filename):
    db_config = {}
    try:
        with open(filename) as myfile:
            for line in myfile:
                name, var = line.partition(":")[::2]
                db_config[name.strip()] = var.strip()
    except FileNotFoundError:
        print(f"Error: {filename} file not found.")
        exit(1)
    return db_config

# Reading the database connection details from the host.txt file
db_config = read_db_config("host.txt")

# Connection string for pyodbc
conn_str = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={db_config['host']},{db_config['port']};"
    f"DATABASE={db_config['database']};"
    f"UID={db_config['user']};"
    f"PWD={db_config['password']}"
)

# Attempting to establish the database connection
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    table_name = "dbo.patient_reports"

    # Executing the SQL query
    cursor.execute(f'SELECT * FROM {table_name}')

    # Fetching all rows from the executed query
    rows = cursor.fetchall()

    # Extracting column names from cursor description
    columns = [column[0] for column in cursor.description]

    # Creating a DataFrame from the fetched data
    df = pd.DataFrame.from_records(rows, columns=columns)

    # Displaying the first few rows of the DataFrame
    print(df.head())

except pyodbc.Error as err:
    print(f"Error: {err}")
    traceback.print_exc()
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    traceback.print_exc()
finally:
    if 'conn' in locals() and conn:
        cursor.close()
        conn.close()


In [ ]:
import pandas as pd
import pyodbc
import traceback
import requests
import gradio as gr

# Ollama server configuration
OLLAMA_BASE_URL = "http://149.163.179.98:11434/api/chat"

# Function to read database configuration from a file
def read_db_config(filename):
    db_config = {}
    try:
        with open(filename) as myfile:
            for line in myfile:
                name, var = line.partition(":")[::2]
                db_config[name.strip()] = var.strip()
    except FileNotFoundError:
        print(f"Error: {filename} file not found.")
        exit(1)
    return db_config

# Function to summarize the reports using the Ollama model
def summarize_text_reports(report_texts, custom_prompt):
    summaries = []
    for text in report_texts:
        try:
            response = requests.post(
                OLLAMA_BASE_URL,
                headers={
                    "Content-Type": "application/json"
                },
                json={
                    "model": "llama3",
                    "messages": [
                        {
                            "role": "user",
                            "content": f"{custom_prompt} {text}"
                        }
                    ]
                },
                stream=True  # Use streaming to handle chunked responses
            )
            
            response.raise_for_status()  # Raise an error for bad responses
            summary_parts = []

            for line in response.iter_lines():
                if line:
                    try:
                        # Parse each line as JSON and extract the message content
                        data = json.loads(line.decode('utf-8'))
                        if "message" in data and "content" in data["message"]:
                            summary_parts.append(data["message"]["content"])
                    except json.JSONDecodeError:
                        print(f"Unable to decode line: {line}")

            # Combine all parts to form the full summary
            summary = ''.join(summary_parts)
            summaries.append(summary)
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            summaries.append("Error: Request failed.")
    
    return "\n\n".join(summaries)

# Function to fetch and summarize reports for a given patient_id
def get_summarized_reports(patient_id, custom_prompt):
    db_config = read_db_config("host.txt")
    
    conn_str = (
        f"DRIVER={{SQL Server}};"
        f"SERVER={db_config['host']},{db_config['port']};"
        f"DATABASE={db_config['database']};"
        f"UID={db_config['user']};"
        f"PWD={db_config['password']}"
    )
    
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        query = f"SELECT report FROM dbo.patient_reports WHERE patient_id = ?"
        
        cursor.execute(query, (patient_id,))
        rows = cursor.fetchall()
        
        if not rows:
            return "No reports found for the given patient ID."

        reports = [row[0] for row in rows]
        summarized_reports = summarize_text_reports(reports, custom_prompt)
        
        return summarized_reports

    except pyodbc.Error as err:
        print(f"Database error: {err}")
        traceback.print_exc()
        return f"Database error: {err}"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        traceback.print_exc()
        return f"An unexpected error occurred: {e}"
    finally:
        if 'conn' in locals() and conn:
            cursor.close()
            conn.close()

# Creating Gradio Interface
def gradio_interface(patient_id, custom_prompt):
    return get_summarized_reports(patient_id, custom_prompt)

iface = gr.Interface(
    fn=gradio_interface,
    inputs=["text", "text"],
    outputs="text",
    title="Patient Report Summarizer",
    description="Enter the patient ID and custom prompt to fetch and summarize their medical reports.",
    examples=[
        ["0000001", "Please summarize the following medical report:"]
    ]
)

if __name__ == "__main__":
    iface.launch()